# EvoJAX NEAT (Colab-Friendly, Live Progress)

This notebook clones/pulls the repo, runs both modes, and exposes real-time progress files.

In [ ]:
import os
import sys
import json
import time
import subprocess
from pathlib import Path

REPO_URL = 'https://github.com/aryangoyal7/NEAT-Sakana.git'
WORKDIR = Path('/content') if Path('/content').exists() else Path.cwd()
REPO_DIR = WORKDIR / 'NEAT-Sakana'

if not REPO_DIR.exists():
    subprocess.run(['git', 'clone', REPO_URL, str(REPO_DIR)], check=True)
else:
    subprocess.run(['git', '-C', str(REPO_DIR), 'pull'], check=True)

SCRIPT_PATH = REPO_DIR / 'scripts' / 'run_evojax_neat_both.py'
OUT_ROOT = REPO_DIR / 'artifacts'
OUT_ROOT.mkdir(parents=True, exist_ok=True)

print('Kernel python:', sys.executable)
print('REPO_DIR    :', REPO_DIR)
print('SCRIPT_PATH :', SCRIPT_PATH)
print('OUT_ROOT    :', OUT_ROOT)

In [ ]:
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install -r "{REPO_DIR / 'requirements.txt'}"
!{sys.executable} -m pip install evojax==0.2.17 flax optax chex orbax-checkpoint tensorstore rich absl-py cma opencv-python-headless
!{sys.executable} -m pip install -e "{REPO_DIR}"

In [ ]:
GENERATIONS = 8
POP_SIZE = 24
MAX_STEPS = 800
EPISODES_DIRECT = 2
EPISODES_SELFPLAY = 1

In [ ]:
env = os.environ.copy()
env['PYTHONPATH'] = f"{REPO_DIR / 'src'}:{env.get('PYTHONPATH','')}"
env['MPLCONFIGDIR'] = '/tmp/mplconfig'
Path('/tmp/mplconfig').mkdir(parents=True, exist_ok=True)

cmd = [
    sys.executable, str(SCRIPT_PATH),
    '--repo-dir', str(REPO_DIR),
    '--generations', str(GENERATIONS),
    '--pop-size', str(POP_SIZE),
    '--max-steps', str(MAX_STEPS),
    '--episodes-direct', str(EPISODES_DIRECT),
    '--episodes-selfplay', str(EPISODES_SELFPLAY),
]

RUN_LOG = OUT_ROOT / f'run_launcher_{int(time.time())}.log'
run_log_f = open(RUN_LOG, 'w', encoding='utf-8')
print('Starting run in background...')
print('Command:', ' '.join(cmd))
print('Launcher log:', RUN_LOG)

run_proc = subprocess.Popen(
    cmd,
    cwd=str(REPO_DIR),
    env=env,
    stdout=run_log_f,
    stderr=subprocess.STDOUT,
    text=True,
)
print('PID:', run_proc.pid)

In [ ]:
def latest_dir(prefix: str):
    dirs = sorted(OUT_ROOT.glob(f'{prefix}_*'), key=lambda p: p.stat().st_mtime, reverse=True)
    return dirs[0] if dirs else None

def print_live(prefix: str):
    d = latest_dir(prefix)
    if d is None:
        print(prefix, '-> waiting for run folder...')
        return
    print(f'{prefix} dir: {d}')
    pj = d / 'progress.json'
    if pj.exists():
        try:
            data = json.loads(pj.read_text())
            print(' progress:', data)
        except Exception as e:
            print(' progress parse error:', e)
    else:
        print(' progress: not created yet')
    hl = d / 'history_live.csv'
    if hl.exists():
        lines = hl.read_text().strip().splitlines()
        if lines:
            print(' history_live last line:', lines[-1])
    sl = d / 'species_sizes_live.csv'
    if sl.exists():
        lines = sl.read_text().strip().splitlines()
        if lines:
            print(' species_live last line:', lines[-1])

print('Monitoring every 30s. Stop this cell manually if needed.')
while run_proc.poll() is None:
    print('\n' + '=' * 100)
    print('time:', time.strftime('%Y-%m-%d %H:%M:%S'))
    print('process running, pid=', run_proc.pid)
    print_live('direct_vs_builtin')
    print_live('selfplay_then_builtin')
    if RUN_LOG.exists():
        tail = RUN_LOG.read_text(encoding='utf-8', errors='ignore').splitlines()[-10:]
        print('--- launcher log tail ---')
        for line in tail:
            print(line)
    time.sleep(30)

print('\nProcess finished with code:', run_proc.returncode)
run_log_f.close()

In [ ]:
if run_proc.poll() is None:
    raise RuntimeError('Process still running. Keep monitor cell running or wait longer.')
if run_proc.returncode != 0:
    tail = RUN_LOG.read_text(encoding='utf-8', errors='ignore').splitlines()[-80:]
    print('\n'.join(tail))
    raise RuntimeError(f'Run failed with code {run_proc.returncode}')

direct_dir = latest_dir('direct_vs_builtin')
selfplay_dir = latest_dir('selfplay_then_builtin')
print('direct_dir  =', direct_dir)
print('selfplay_dir=', selfplay_dir)
print('launcher log=', RUN_LOG)

In [ ]:
from IPython.display import display, Markdown, Image

def show_img(path: Path, width=700):
    if path.exists():
        display(Markdown(f'`{path}`'))
        display(Image(filename=str(path), width=width))
    else:
        print('Missing:', path)

show_img(direct_dir / 'plots' / 'fitness_complexity.png')
show_img(direct_dir / 'plots' / 'species_sizes.png')
show_img(direct_dir / 'plots' / 'champion_network.png')
show_img(direct_dir / 'gifs' / 'champion_vs_builtin.gif', width=520)

show_img(selfplay_dir / 'plots' / 'fitness_complexity.png')
show_img(selfplay_dir / 'plots' / 'species_sizes.png')
show_img(selfplay_dir / 'plots' / 'champion_network.png')
show_img(selfplay_dir / 'gifs' / 'champion_vs_builtin.gif', width=520)
show_img(selfplay_dir / 'gifs' / 'champion_vs_runnerup.gif', width=520)

In [ ]:
for p in [direct_dir / 'report.md', selfplay_dir / 'report.md']:
    print('\n' + '=' * 100)
    print(p)
    print('=' * 100)
    if p.exists():
        print(p.read_text(encoding='utf-8'))
    else:
        print('Missing report file')